### Going to add in MS DRG codes, length of stay, patient age (in this dataset, patients over 89 will show as 300+ due to the way records were de-identified), ICD-9 and ICD-10 code

In [2]:
import sqlite3 as sl
import pandas as pd

#connect to sql
conn = sl.connect('../Data/patient_data.db')
c = conn.cursor()

In [3]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print (name[0])

ADMISSIONS
CALLOUT
CPTEVENTS
DX_ICD
D_CPT
DRGCODES
ICUSTAY
NOTES
PATIENTS
SERVICES
DRG


### used https://www.aapc.com/icd-10/codes/ converter to get the correct icd10 code.

In [6]:
new_data = pd.read_sql('''
                    SELECT adm.subject_id, 
                          adm.hadm_id,
                          admission_type,
                          admission_location,
                          notes.chartdate,
                          notes.charttime,
                          notes.category,
                          notes.description,
                          notes.text,
                          adm.diagnosis,
                          cast(JulianDay(adm.dischtime) - JulianDay(adm.admittime) as int) as LOS,
                          drg.DRG_CODE,
                          drg.DRG_TYPE,
                          drg.DESCRIPTION AS DRG_DESC,
                          drg.DRG_SEVERITY,
                          drg.DRG_MORTALITY,
                          dx.icd9_code,
                          CASE WHEN dx.icd9_code = 53100 THEN 'K25.0'
                               WHEN dx.icd9_code = 5789 THEN 'K92.2'
                               WHEN dx.icd9_code = 4019 THEN 'I16.9'
                               WHEN dx.icd9_code = 42731 THEN 'I48.91'
                               WHEN dx.icd9_code = 431 THEN 'I61.9'
                               else 'other' end as 'icd10_code',
                         pt.gender,
                         pt.dob,
                         pt.dod,
                         pt.dod_hosp,
                         pt.expire_flag
                    FROM admissions as adm
                    JOIN notes on adm.subject_id = notes.subject_id
                         AND adm.hadm_id = notes.hadm_id
                    JOIN drg on drg.subject_id = adm.subject_id and drg.hadm_id = adm.hadm_id
                    JOIN DX_ICD dx on dx.subject_id = adm.subject_id and dx.hadm_id = adm.hadm_id
                    JOIN patients pt on pt.subject_id = adm.subject_id 
                    WHERE dx.icd9_code in ('53100','5789','4019','42731','431')
                    AND drg.drg_type = 'MS'
                    AND notes.category in ('Discharge summary','Physician');
                       ''',conn)
new_data.to_csv('../Data/new_df.csv')
new_data

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,CHARTDATE,CHARTTIME,CATEGORY,DESCRIPTION,TEXT,DIAGNOSIS,...,DRG_DESC,DRG_SEVERITY,DRG_MORTALITY,ICD9_CODE,icd10_code,GENDER,DOB,DOD,DOD_HOSP,EXPIRE_FLAG
0,5689,157267,EMERGENCY,EMERGENCY ROOM ADMIT,2124-09-28,None,Discharge summary,Report,Admission Date: [**2124-9-18**] ...,EKG CHANGES,...,"ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS...",None,None,4019,I16.9,F,2048-07-14 00:00:00,2125-01-08 00:00:00,2125-01-08 00:00:00,1
1,5689,157267,EMERGENCY,EMERGENCY ROOM ADMIT,2124-09-28,None,Discharge summary,Report,Admission Date: [**2124-9-18**] ...,EKG CHANGES,...,"ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS...",None,None,42731,I48.91,F,2048-07-14 00:00:00,2125-01-08 00:00:00,2125-01-08 00:00:00,1
2,28389,139931,EMERGENCY,EMERGENCY ROOM ADMIT,2152-09-25,None,Discharge summary,Report,Admission Date: [**2152-9-15**] ...,ABDOMINAL PAIN;AORTIC DISSECTION;TELEMETRY,...,MAJOR SMALL & LARGE BOWEL PROCEDURES W MCC,None,None,4019,I16.9,F,2104-11-22 00:00:00,None,None,0
3,16072,198295,EMERGENCY,EMERGENCY ROOM ADMIT,2184-11-24,None,Discharge summary,Report,Admission Date: [**2184-11-19**] ...,ATRIAL FIBRILLATION;RESPIRATORY FAILURE,...,SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,None,None,4019,I16.9,F,2143-11-23 00:00:00,2184-11-24 00:00:00,2184-11-24 00:00:00,1
4,16072,198295,EMERGENCY,EMERGENCY ROOM ADMIT,2184-11-24,None,Discharge summary,Report,Admission Date: [**2184-11-19**] ...,ATRIAL FIBRILLATION;RESPIRATORY FAILURE,...,SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,None,None,42731,I48.91,F,2143-11-23 00:00:00,2184-11-24 00:00:00,2184-11-24 00:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20856,60929,109316,EMERGENCY,CLINIC REFERRAL/PREMATURE,2142-07-04,None,Discharge summary,Report,Admission Date: [**2142-6-29**] ...,SEIZURE,...,ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REHABILIT...,None,None,4019,I16.9,F,2084-06-29 00:00:00,None,None,0
20857,75779,123505,EMERGENCY,CLINIC REFERRAL/PREMATURE,2128-08-27,None,Discharge summary,Report,Admission Date: [**2128-8-6**] D...,CONGESTIVE HEART FAILURE,...,CARDIAC VALVE & OTH MAJ CARDIOTHORACIC PROC W ...,None,None,4019,I16.9,M,2055-06-23 00:00:00,None,None,0
20858,75779,123505,EMERGENCY,CLINIC REFERRAL/PREMATURE,2128-08-27,None,Discharge summary,Report,Admission Date: [**2128-8-6**] D...,CONGESTIVE HEART FAILURE,...,CARDIAC VALVE & OTH MAJ CARDIOTHORACIC PROC W ...,None,None,42731,I48.91,M,2055-06-23 00:00:00,None,None,0
20859,46449,110075,EMERGENCY,CLINIC REFERRAL/PREMATURE,2174-06-07,None,Discharge summary,Addendum,"Name: [**Known lastname 13679**],[**Known fir...",CHEST PAIN,...,CORONARY BYPASS W CARDIAC CATH W/O MCC,None,None,42731,I48.91,M,2100-02-13 00:00:00,None,None,0


In [7]:
new_data.icd10_code.value_counts()

I16.9     11728
I48.91     7587
I61.9       802
K92.2       722
K25.0        22
Name: icd10_code, dtype: int64

In [8]:
new_data.DRG_CODE.value_counts()

236    926
871    865
64     773
220    773
219    529
      ... 
455      1
503      1
615      1
711      1
708      1
Name: DRG_CODE, Length: 549, dtype: int64